<a href="https://colab.research.google.com/github/agentinali/hello/blob/master/CTC_Loss_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!pip install captcha
from keras.models import *
from keras.layers import *
#from captcha.image import ImageCaptcha
import matplotlib.pyplot as plt
import numpy as np
import random
from keras import backend as K
import string
import keras
import tensorflow as tf


config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 4} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

#K.tensorflow_backend._get_available_gpus()

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

def merge(inputs, mode, concat_axis=-1):
    return concatenate(inputs, concat_axis)


def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    y_pred = y_pred[:, 2:, :]
    print(y_pred)
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)



characters = string.digits + string.ascii_uppercase  #必須包含空白
print(characters)

#Saw a non-null label (index >= num_classes - 1) following a null label
#您必須指定類的數量比您擁有的字符數多一個。
width, height, n_len, n_class = 175, 78, 7, len(characters)+1  
# width, height, n_len, n_class = 170, 80, 6, len(characters)+1  

rnn_size = 1024

#建立rnn模型

input_tensor = Input((width, height, 3))
# input_tensor = Input((width, height,1)) #灰階
x = input_tensor
for i in range(3):
    x = Convolution2D(32, (3, 3), activation='relu')(x)
    x = Convolution2D(32, (3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

conv_shape = x.get_shape()
x = Reshape(target_shape=(int(conv_shape[1]), int(conv_shape[2]*conv_shape[3])))(x)
x = Dense(64, activation='relu')(x)

# gru_0 =  GRU(rnn_size, return_sequences=True, kernel_initializer='he_normal', name='gru0')(x)
# gru_0b = GRU(rnn_size, return_sequences=True, go_backwards=True, 
#              kernel_initializer='he_normal', name='gru0_b')(x)
# x= add([gru_0, gru_0b])

gru_1 =  GRU(rnn_size, return_sequences=True, kernel_initializer='he_normal', name='gru1')(x)
gru_1b = GRU(rnn_size, return_sequences=True, go_backwards=True, 
             kernel_initializer='he_normal', name='gru1_b')(x)
gru1_merged = add([gru_1, gru_1b])

gru_2 =  GRU(rnn_size, return_sequences=True, kernel_initializer='he_normal', name='gru2')(gru1_merged)
gru_2b = GRU(rnn_size, return_sequences=True, go_backwards=True, 
             kernel_initializer='he_normal', name='gru2_b')(gru1_merged)
x= concatenate([gru_2, gru_2b])

x = Dropout(0.25)(x)

# x = Dense(512, activation='relu')(x)
x = Dense(n_class,  activation='softmax', kernel_initializer="he_normal")(x)
base_model = Model(inputs=input_tensor, outputs=x)
print(base_model.summary())

labels = Input(name='the_labels', shape=[n_len], dtype='float32')
input_length = Input(name='input_length', shape=[1], dtype='int64')
label_length = Input(name='label_length', shape=[1], dtype='int64')
loss_out = Lambda(ctc_lambda_func, output_shape=(1,), 
                  name='ctc')([x, labels, input_length, label_length])

model = Model(inputs=[input_tensor, labels, input_length, label_length], outputs=[loss_out])
# model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer='Adam')
model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer='adadelta')

from keras.utils.vis_utils import plot_model
from IPython.display import Image

plot_model(model, to_file="model.png", show_shapes=True)
Image('model.png')


In [0]:
!pip install captcha

In [0]:
#自動產生車牌函數
def gencar(batch_size=32):
    X = np.zeros((batch_size, width, height,3), dtype=np.float32)
#     X = np.zeros((batch_size, width, height, 1), dtype=np.float32) #灰階
    y = np.zeros((batch_size, n_len), dtype=np.uint8)
    while True:
        l_plateStr,l_plateImg = G.genBatch(batch_size, 34, './plate_test', (175,78))
        
        for i in range(batch_size):
#             x1=l_plateImg[i].reshape(78,175, 1)  #灰階
#             X[i] = np.array(x1).transpose(1,0,2)/255.0 #灰階
            X[i] = np.array(l_plateImg[i]).transpose(1,0,2)/255.0
            #np.array(x).transpose(1, 0, 2)  0，1，2  => 1，0，2
            #代表是將圖像3維w*h*c => h*w*c   w  h  c     h  w  c                                            
            y[i] = [characters.find(x) for x in l_plateStr[i]]
           
        yield [X, y, np.ones(batch_size)*int(conv_shape[1]-2),
                 np.ones(batch_size)*n_len], np.ones(batch_size)



In [0]:

# generator = gencar()
# [X_test, y_test, _, _], _  = next(generator)


acc = evaluate(base_model)*100
#         self.accs.append(acc)
print('acc: %f%%'%acc)


In [0]:
#計算識別率
def evaluate(model, batch_num=64):
    batch_acc = 0
    generator = gencar()
      
    for i in range(batch_num):
        
        [X_test, y_test, _, _], _  = next(generator)
        y_pred = base_model.predict(X_test)
        shape = y_pred[:,2:,:].shape
        ctc_decode = K.ctc_decode(y_pred[:,2:,:], 
                                  input_length=np.ones(shape[0])*shape[1])[0][0]
        out = K.get_value(ctc_decode)[:, :7]
        if out.shape[1] == 7:
            batch_acc += ((y_test == out).sum(axis=1) == 7).mean()
       
    return batch_acc / batch_num



In [0]:
from keras.callbacks import *

class Evaluate(Callback):
    def __init__(self):
        self.accs = []
    
    def on_epoch_end(self, epoch, logs=None):
        acc = evaluate(base_model)*100
        self.accs.append(acc)
        print('acc: %f%%'%acc)

evaluator = Evaluate()

In [0]:
best_model = ModelCheckpoint("mytrain_best.h5", monitor='val_loss', verbose=1, save_best_only=True)
train_history=model.fit_generator(gencar(32), steps_per_epoch=500, epochs=20,
#                      callbacks=[EarlyStopping(patience=10),best_model],
                    callbacks=[EarlyStopping(patience=10),best_model, evaluator],
                    validation_data=gencar(32), validation_steps=250)

#print(train_history)

In [0]:
#测试模型
from PIL import Image
characters2 = characters + ' '
n_len=7
batch_acc = 0



#圖形灰階預處理
# img= cv2.imread("./gdrive/My Drive/Car_regression/testjpg/59229.jpg", 0)
# ##img = Image.open("./gdrive/My Drive/Car_regression/testjpg/fg_gane11.jpg").convert("L")
# img = cv2.resize(np.array(img),(175,78))
# ##img= cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# plt.imshow(img)
# print(img.shape)
# img= img.reshape(78,175, 1)
# X = np.zeros((1, width, height, 1), dtype=np.float32)
# end 圖形灰階

# 彩色3通道
jpglist = glob( "./gdrive/My Drive/Car_regression/testjpg/59229.jpg" )
img = cv2.imread("./gdrive/My Drive/Car_regression/testjpg/abc5678.jpg")
img = cv2.imread("fg_gane11.jpg")
# img = cv2.imread(jpglist[0])
img = img[:,:, (2, 1, 0)]
#img = cv2.resize(img,(380,160),interpolation=cv2.INTER_AREA)
img = cv2.resize(img,(175,78))
# img = cv2.resize(np.array(img),(175,78))
#img = img[:,:, (2, 1, 0)]
X = np.zeros((len(jpglist), width, height, 3), dtype=np.float32)

# #y = np.zeros((len(jpg), 16), dtype=np.uint8)
# end 彩色3通道
plt.imshow(img)

#plt.imshow(img)
#print(img.shape)
X_test=X
#print(X_test.shape)
#print(X.shape)
X_test[0]=np.array(img).transpose(1,0, 2)/250.0

#print(X_test.shape)

# for i,jpg in  enumerate(jpglist):
# #     img = Image.open(jpg).convert("RGB")
# #     img = img.resize((175,78), Image.ANTIALIAS)
#     im = cv2.imread(jpg)
#     img = cv2.resize(im,(175,78));
#     X_test[i]=np.array(img).transpose(1, 0, 2)/255
# plt.imshow(img)

#for i in range(10):
#[X_test, y_test, _, _], _  = next(gencar())
y_pred = base_model.predict(X_test)
# print(y_pred)
y_pred = y_pred[:,2:,:]

out = K.get_value(K.ctc_decode(y_pred, input_length=np.ones(y_pred.shape[0])*y_pred.shape[1])[0][0])[:, :7]

print(out)
# print(y_test)
# batch_acc += ((y_test == out).sum(axis=1) == 7).mean()
# print(batch_acc)
# for i in range(32)    :
#     out1 = ''.join([characters[x] for x in out[i]])
#     y_true = ''.join([characters[x] for x in y_test[i]])
#     plt.figure(i)
#     plt.imshow(X_test[i].transpose(1, 0, 2))
#     plt.title('pred:' + str(out1) + '\ntrue: ' + str(y_true))

argmax = np.argmax(y_pred, axis=2)[0]
print(argmax)
list(zip(argmax, ''.join([characters2[x] for x in argmax])))
#print(aa)



In [0]:
generator = ImageCaptcha(width=width, height=height)
random_str = 'O0O0'
X = generator.generate_image(random_str)
X = np.expand_dims(X, 0)

y_pred = model.predict(X)
plt.title('real: %s\npred:%s'%(random_str, decode(y_pred)))
plt.imshow(X[0], cmap='gray')

In [0]:


第一種方式:

import cv2

image = cv2.imread('test.jpg') #讀取影像

image_g = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) #透過轉換函式轉為灰階影像

cv2.imwrite('test_g.jpg', image_g) #儲存影像

 

第二種方式:

import cv2

image_g = cv2.imread('test.jpg', 0) #直接以灰階方式讀取影像

cv2.imwrite('test_g.jpg', image_g) #儲存影像

X_train.reshape(X_train.shape[0], 28, 28, 1).astype(‘float32’) 

※查詢OpenCV應用目錄總整理如下※

http://https://jennaweng0621.pixnet.net/blog/post/403630133

※查詢其他OpenCV用法如下※

http://jennaweng0621.pixnet.net/blog/category/6758024
    
    
    
img = img[:,:, (2, 1, 0)] # 改变图像通道顺序为：RGB


# 由于cv2.imread得到的图像是按照BGR进行排列的，而我们经常需要RGB的图像，因此需要先分离图像通道，然后再融合图像
b,g,r = cv2.split(img)
# 通过split来分离通道
b0 = img[:,:,0]
# 也可以通过切片操作来进行得到
# 注意split是一个比较耗时的操作，尽量不要使用
img = cv2.merge((r,g,b))


流程大概會是：

影像讀取 -- 調整圖像大小 -- 影像去噪 -- 圖像閾值處理 -- 膨脹框出圖像中的內容的輪廓 --尋找與身份證字號相似的大小輪廓 -- Tesseract文字辨識

用到的工具：PIL、pytesseract、cv2、numpy

from PIL import Image #version 3.2.0
import pytesseract as ocr #version 0.2.5
import cv2 #version 3.4.5
import numpy as np #version 1.15.2
1.影像讀取

這裡把影片讀取進來格式以預設「IMREAD_COLOR」
IMREAD_COLOR：讀入RGB三個波段格式
IMREAD_GRAYSCALE：讀入灰階格式
IMREAD_UNCHANGED：讀入原始圖片格式

#1.讀取影像
imgPath="image2.jpg"
img = cv2.imread(imgPath, cv2.IMREAD_COLOR)
2.調整圖像大小

改變尺吋:interpolation內插方式
CV_INTER_NEAREST ：最鄰近插點法。
CV_INTER_LINEAR ：雙線性插補(預設)。
CV_INTER_AREA ：臨域像素再取樣插補。縮小用
CV_INTER_CUBIC ：雙立方插補，4×4大小的補點。放大用
CV_INTER_LANCZOS4 ：Lanczos插補，8×8大小的補點。

#2.調整圖像大小
img = cv2.resize(img, (428, 270), interpolation=cv2.INTER_CUBIC)


3.影像去噪

彩色圖片的噪點去除，並且拍照時可能會有歪歪斜斜的可能，需要轉正圖片

#3.影像去噪
gray = cv2.fastNlMeansDenoisingColored(img, None, 10, 3, 3, 3)
coefficients = [0, 1, 1]
m = np.array(coefficients).reshape((1, 3))
#旋轉圖片
gray = cv2.transform(gray, m)

4.圖像閾值處理

閾值處理產生二值化圖片，框出文字區域，讓後面做形態轉變(澎脹或是腐蝕...)

#4.閾值 180  maxval:255
ret, binary = cv2.threshold(gray, 180, 255, cv2.THRESH_BINARY_INV)
ele = cv2.getStructuringElement(cv2.MORPH_RECT, (15, 10))
5.膨脹文字區域

給二值化後的圖片與文字框們，將它們一一做膨脹處理

#5.膨脹操作
dilation = cv2.dilate(binary, ele, iterations=1)
6.尋找與身份證字號相似的大小輪廓

尋找之外，也把所找到的文子區域在原圖中框出來

#6.尋找與身份證字號相似的大小輪廓(contours)
image, contours, hierarchy = cv2.findContours(dilation, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
for box in contours:  
        i_index+=1
        h = abs(box[0][1] - box[2][1])
        w = abs(box[0][0] - box[2][0])
        max_point=box[3][0]
        Xs = [i[0] for i in box]
        Ys = [i[1] for i in box]
        x1 = min(Xs)
        y1 = min(Ys)
        #找到的輪廓用緣色的框畫出來
        cv2.drawContours(img, [box], 0, (0, 255, 0), 2)
        img_gray=img[y1:y1 + h, x1:x1+w]
        # 取得每一個小輪廓二值化圖片
        idImg = cv2.resize(img_gray, (img_gray.shape[1] * 3, img_gray.shape[0] * 3), interpolation=cv2.INTER_CUBIC)
        idImg = cv2.cvtColor(gray, cv2.COLOR_BGR2GRAY)
        #otsu二值化操作
        retval, idImg = cv2.threshold(idImg , 120, 255, cv2.THRESH_OTSU + cv2.THRESH_BINARY)
        
        print("index:",str(i_index),"tag:",box,"high:",h,"width:",w)
        #抓到哪些輪廓
        cv2.imwrite("data\idImg_"+str(i_index)+".png", idImg)
        cv2.imwrite("data\contours.png", img)    
        #x軸在最右邊，且輪廓大於4倍視為身份證字號位置
        if (max_point>max_item and w//h>=4.0):
            max_item=max_point
            max_index=i_index



In [0]:
#新版台灣車牌產生器

#coding=utf-8
import PIL
from PIL import ImageFont
from PIL import Image
from PIL import ImageDraw
import cv2;
import numpy as np;
import os;
from math import *
from glob import glob
from os.path import splitext
import random
import matplotlib.pyplot as plt

# from keras.models import Model
# from keras.callbacks import ModelCheckpoint
# from keras.layers import Conv2D, MaxPool2D, Flatten, Dropout, Dense, Input, BatchNormalization
# from keras.optimizers import Adam
# from keras.backend.tensorflow_backend import set_session
# from keras.utils.vis_utils import model_to_dot
# import tensorflow as tf
# import matplotlib
# import matplotlib.pyplot as plt
# from matplotlib.font_manager import FontProperties
# from IPython.display import SVG
#model.add(layers.BatchNormalization())

# font = ImageFont.truetype("Arial-Bold.ttf",14)

chars = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z"];
chars7 = ["0", "1", "2", "3", "5", "6", "7", "8", "9", "A", "B", "C", "D", "E", "F", "G", "H", "J", "K", "L", "M", "N", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z"];
BGPATH7='./gdrive/My Drive/Car_regression/images/template-2.jpg'
BGPATH6='./gdrive/My Drive/Car_regression/images/template-3.png'
CODE7='./gdrive/My Drive/Car_regression/font/platechar.ttf'
CODE6='./gdrive/My Drive/Car_regression/font/msgothic.ttc'
basepath='./gdrive/My Drive/Car_regression/'
def AddSmudginess(img, Smu):
    rows = r(Smu.shape[0] - 50)
    cols = r(Smu.shape[1] - 50)
    adder = Smu[rows:rows + 50, cols:cols + 50];
    adder = cv2.resize(adder, (50, 50));
    #   adder = cv2.bitwise_not(adder)
    img = cv2.resize(img,(50,50))
    img = cv2.bitwise_not(img)
    img = cv2.bitwise_and(adder, img)
    img = cv2.bitwise_not(img)
    return img
  

def rot(img,angel,shape,max_angel):
    """ 使图像轻微的畸变

        img 输入图像
        factor 畸变的参数
        size 为图片的目标尺寸

    """
    size_o = [shape[1],shape[0]]

    size = (shape[1]+ int(shape[0]*cos((float(max_angel )/180) * 3.14)),shape[0])


    interval = abs( int( sin((float(angel) /180) * 3.14)* shape[0]));

    pts1 = np.float32([[0,0]         ,[0,size_o[1]],[size_o[0],0],[size_o[0],size_o[1]]])
    if(angel>0):

        pts2 = np.float32([[interval,0],[0,size[1]  ],[size[0],0  ],[size[0]-interval,size_o[1]]])
    else:
        pts2 = np.float32([[0,0],[interval,size[1]  ],[size[0]-interval,0  ],[size[0],size_o[1]]])

    M  = cv2.getPerspectiveTransform(pts1,pts2);
    dst = cv2.warpPerspective(img,M,size);

    return dst;

def rotRandrom(img, factor, size):
 
    """
    添加放射畸变
    img 输入图像
    factor 畸变的参数
    size 为图片的目标尺寸
    """
    shape = size
    pts1 = np.float32([[0, 0], [0, shape[0]], [shape[1], 0], [shape[1], shape[0]]])
    pts2 = np.float32([[r(factor), r(factor)], [ r(factor), shape[0] - r(factor)], [shape[1] - r(factor),  r(factor)],
                       [shape[1] - r(factor), shape[0] - r(factor)]])
    M = cv2.getPerspectiveTransform(pts1, pts2)
    dst = cv2.warpPerspective(img, M, size)
    return dst



def tfactor(img):
    """
    添加饱和度光照的噪声
    """
    hsv = cv2.cvtColor(img,cv2.COLOR_BGR2HSV);
    hsv[:,:,0] = hsv[:,:,0]*(0.8+ np.random.random()*0.2);
    hsv[:,:,1] = hsv[:,:,1]*(0.3+ np.random.random()*0.7);
    hsv[:,:,2] = hsv[:,:,2]*(0.2+ np.random.random()*0.8);

    img = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR);
    return img

def random_envirment(img,data_set):
    """
    添加自然环境的噪声    
    """
    index=r(len(data_set))
    env = cv2.imread(data_set[index])
    env = cv2.cvtColor(env, cv2.COLOR_BGR2RGB)
    env = cv2.resize(env,(img.shape[1],img.shape[0]))

    bak = (img==0);
    bak = bak.astype(np.uint8)*255;
    inv = cv2.bitwise_and(bak,env)
    img = cv2.bitwise_or(inv,img)
    return img


def AddGauss(img, level):
    """
    添加高斯模糊
    """
    return cv2.blur(img, (level * 2 + 1, level * 2 + 1));

def r(val):
    return int(np.random.random() * val)

def AddNoiseSingleChannel(single):
    """
    添加高斯噪声
    """
    diff = 255-single.max();
    noise = np.random.normal(0,1+r(6),single.shape);
    noise = (noise - noise.min())/(noise.max()-noise.min())
    noise= diff*noise;
    noise= noise.astype(np.uint8)
    dst = single + noise
    return dst

def addNoise(img,sdev = 0.5,avg=10):
    img[:,:,0] =  AddNoiseSingleChannel(img[:,:,0]);
    img[:,:,1] =  AddNoiseSingleChannel(img[:,:,1]);
    img[:,:,2] =  AddNoiseSingleChannel(img[:,:,2]);
    return img;


class GenPlate:
    def __init__(self,NoPlates,fontEng7=CODE7,fontEng6=CODE6):
        self.buddyp =  30
        self.fontE7 =  ImageFont.truetype(fontEng7,110,0)
        self.fontE6 =  ImageFont.truetype(fontEng6,100,0)
        self.fontdt =  ImageFont.truetype(fontEng6,100,0)
        self.bg7  = Image.open(BGPATH7).convert("RGB")
        self.bg6  = Image.open(BGPATH6).convert("RGB")
        self.smu = cv2.imread("./gdrive/My Drive/Car_regression/images/smu2.jpg");
        self.noplates_path = [];
        for parent,parent_folder,filenames in os.walk(NoPlates):
            for filename in filenames:
                path = parent+"/"+filename;
                self.noplates_path.append(path);



    def draw(self,val,gencode):
    
        """    
          生成英文字符
    
        """
        pt1=0
        if gencode == 24 or gencode == 42:
            img = self.bg6
            img = img.resize((320,150), Image.ANTIALIAS)
            draw = ImageDraw.Draw(img)
            for i in range(len(val)):
                draw.text((15+i*45+pt1, self.buddyp),val[i],(0,0,0),font=self.fontE6)
                if i==1 and gencode == 24: 
                    pt1=15
                    draw.text((100, 0),'.',(0,0,0),font=self.fontdt)
                if i==3 and gencode == 42:       
                    pt1=15
                    draw.text((190, 0),'.',(0,0,0),font=self.fontdt)     

        if gencode == 34:
            img = self.bg7
            img = img.resize((380,160), Image.ANTIALIAS)
            draw = ImageDraw.Draw(img)
            for i in range(len(val)):
                draw.text((20+i*44+pt1, self.buddyp),val[i],(0,0,0),font=self.fontE7)
                if i==2:
                    pt1=15
                    draw.text((155, 0),'.',(0,0,0),font=self.fontdt) 

        return img
                
        
       
    def generate(self,text,gencode):
      
        fg = self.draw(text,gencode) 
        com=np.array(fg)
        cv2.imwrite('fg_gane11.jpg',cv2.cvtColor(com, cv2.COLOR_RGB2BGR))
        
       
        com_r = rot(com,r(40)-20,com.shape,20);
        com_r = rotRandrom(com_r,10,(com_r.shape[1],com_r.shape[0]));
        com_r = AddSmudginess(com_r,self.smu)

            # com = tfactor(com)
        com_r = random_envirment(com_r,self.noplates_path)
        com_r = AddGauss(com_r, 1+r(2))
        com_r = addNoise(com_r)
        
        
        
        
        if gencode == 42  or gencode == 24: 
            com = cv2.resize(com,(320,150))
            com_r = cv2.resize(com_r,(320,150))
        if gencode == 34:   
            com = cv2.resize(com,(380,160))
            com_r = cv2.resize(com_r,(380,160))
#        cv2.imwrite('fg_gane17.jpg',cv2.cvtColor(com, cv2.COLOR_BGR2GRAY))
#        print(com.shape)
        
#        com =cv2.cvtColor(com, cv2.COLOR_BGR2GRAY)
#         img =  cv2.resize(com,(175,78));
#         cv2.imwrite('fff.jpg',img)
#        print(com.shape)
        return com,com_r        
  
  

    def genPlateString(self,gencode):
        '''
	 生成车牌String,存为图片
         生成车牌list,存为label
        '''
        plateStr ='';
        plateStr1 ='';
        if gencode==42:
            b = np.random.choice(chars[0:10], 4)
            plateStr+=''.join([e for e in b])
            b = np.random.choice(chars[10:], 1)
            plateStr+=''.join([e for e in b])
            b = np.random.choice(chars, 1)
            plateStr+=''.join([e for e in b])
        if gencode==24:
            b = np.random.choice(chars[10:], 1)
            plateStr+=''.join([e for e in b])
            b = np.random.choice(chars, 1)
            plateStr+=''.join([e for e in b])      
            b = np.random.choice(chars[0:10], 4)
            plateStr+=''.join([e for e in b])
        if gencode==34:   #個人汽車
            b = np.random.choice(chars7[9:], 3)
            plateStr+=''.join([e for e in b])
            b = np.random.choice(chars7[0:9], 4)
            plateStr+=''.join([e for e in b])        
#         if gencode==33:   #營業用車
#             b = np.random.choice(chars7[9:], 3)
#             plateStr+=''.join([e for e in b])
#             b = np.random.choice(chars7[0:9], 3)
#             plateStr+=''.join([e for e in b])            
#         if  plateStr =='':
#             plateStr ='ERR999'
            
        plateStr1=plateStr    
        return plateStr,plateStr1    

    

    def genBatch(self, batchSize,gencode, outputPath,size):
        l_plateStr=[]
        l_plateImg=[]
        if (not os.path.exists(outputPath)):
              os.mkdir(outputPath)
   
        if (gencode==42 or gencode==24): self.buddyp = 30
        if (gencode==34) : self.buddyp = 20    
        for i in range(batchSize):
            plateStr,plateStr1 = G.genPlateString(gencode)
            l_plateStr.append(plateStr)
            img,img_r =  G.generate(plateStr,gencode);
            img =  cv2.resize(img,size);
            img_r =  cv2.resize(img_r,size);
            filename = os.path.join(outputPath, str(i).zfill(4) + '.' + plateStr + ".jpg")
            a=0
#             a=random.choices([1,2])
#             if a==1:
#                 l_plateImg.append(img_r)
#             else:
            l_plateImg.append(img)
#            cv2.imwrite('fff.jpg',img_r)

        return l_plateStr,l_plateImg


#G = GenPlate('./NoPlates')
if __name__=='__main__':
#虛擬機和自己的Google Drive連接
    from google.colab import drive
    drive.mount('/content/gdrive/')
    import os
#     os.chdir("./")

#     !pwd
#     !rm *.jpg
#     !mkdir temp


# G = GenPlate('./NoPlates','./gdrive/My Drive/Car_regression/font/platechar.ttf','./gdrive/My Drive/Car_regression/font/msgothic.ttc')
    G = GenPlate('./gdrive/My Drive/Car_regression/NoPlates')
    a,b=G.genBatch(2,34,'./plate_test',(175,78))    # 34 前3英文後4數字  42 前4數字後2英數   24 前2英數後4數字
#     image= cv2.imread("fff.jpg",0)
#     image = Image.open("fff.jpg").convert("L")


#     print(type(image)) #显示图片类型 numpy类型的数组
#      #图像矩阵的shape属性表示图像的大小，shape会返回tuple元组，第一个元素表示矩阵行数，第二个元组表示矩阵列数，第三个元素是3，表示像素值由光的三原色组成
#     print(image.shape)
#     print(image.size) #图像大小
#     print(image.dtype) #图像类型
#     pixel_data = np.array(image)
#     print(pixel_data)#图片矩阵



#     aa=np.array(img)
#     bb=np.array(b[0])
#     print(aa.shape)
#     # print('-----------------')
#     print(bb.shape)

#     (aa==bb).all()
#     cc=list(aa==bb)
#     print(cc)
#     a=0
#     b=175*78
#     print(len(cc))
#     for i in range(len(cc)):
#         if i: a+=1 
#     print(a,b)        
#     plt.imshow(img)

